# Mars image recognition

#### Mount drive data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Includes

In [3]:
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

#### Load data from files

In [ ]:
data_folder = '../data/hirise-map-proj-v3_2/'

In [ ]:
labels_names = [
    'other', 'crater', 'dark dune', 'slope streak', 'bright dune',
    'impact ejecta', 'swiss cheese', 'spider'
]

In [ ]:
def extract_metadata() -> pd.DataFrame:
    images_and_labels = None
    with open(data_folder + 'labels-map-proj_v3_2_train_val_test.txt',
              'r') as f:
        images_and_labels = f.readlines()
    images_and_labels = pd.DataFrame(images_and_labels).replace(
        '\n', '', regex=True)[0].str.split(' ', expand=True)
    images_and_labels.rename(columns={
        0: 'file_name',
        1: 'label',
        2: 'type'
    },
                             inplace=True)
    return images_and_labels

In [ ]:
metadata = extract_metadata()

In [ ]:
train_data = metadata[metadata['type'] == 'train']
test_data = metadata[metadata['type'] == 'test']
val_data = metadata[metadata['type'] == 'val']

In [ ]:
def image_to_np(image: pd.Series) -> np.array:
    return np.asarray(
        PIL.Image.open(data_folder + 'map-proj-v3_2/' + image['file_name']), )

In [ ]:
def get_data(data: pd.DataFrame) -> tuple:
    labels = data['label']
    data = data.apply(image_to_np, axis=1)
    return np.stack(data.to_numpy()), np.stack(
        labels.to_numpy()).astype('float32')

In [ ]:
train_data, train_labels = get_data(train_data)
test_data, test_labels = get_data(test_data)
val_data, val_labels = get_data(val_data)